## Install DGL-KE
Before training the model, we need to install dgl and dgl-ke packages as well as other dependencies. 

In [ ]:
!pip3 install torch
!pip3 install dgl==0.4.3post2 
!pip3 install dglke

## Prepare train/valid/test set
Before training, we need to split the original drkg into train/valid/test set with a 9:0.5:0.5 manner.

In [ ]:
import pandas as pd
import numpy as np
import sys
import dglke
drkg_file = '../data/drkg.tsv'
df = pd.read_csv(drkg_file, sep="\t", header=None)
triples = df.values.tolist()

In [15]:
# Please make sure the output directory exist.
seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
valid_cnt = int(num_triples * 0.05)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
valid_set = seed[train_cnt:train_cnt+valid_cnt].tolist()
test_set = seed[train_cnt+valid_cnt:].tolist()

with open("train/drkg_train.tsv", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
with open("train/drkg_valid.tsv", 'w+') as f:
    for idx in valid_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("train/drkg_test.tsv", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

In [2]:
!DGLBACKEND=pytorch dglke_train --dataset DRKG --data_path ./train --data_files drkg_train.tsv drkg_valid.tsv drkg_test.tsv --format 'raw_udd_hrt' --model_name TransR --batch_size 2048 \
--neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 1 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --gpu 0 1 --num_proc 2 --neg_sample_size_eval 10000 --async_update

Using backend: pytorch
Reading train triples....
Finished. Read 5286834 train triples.
Reading valid triples....
Finished. Read 293713 valid triples.
Reading test triples....
Finished. Read 293714 test triples.
|Train|: 5286834
random partition 5286834 edges into 2 parts
part 0 has 2643417 edges
part 1 has 2643417 edges
/home/pub_data/hxzhao/miniconda3/envs/dgl/lib/python3.8/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
|valid|: 293713
|test|: 293714
Total initialize time 18.973 seconds
proc 0 takes 7.995 seconds
proc 1 takes 9.286 seconds
training takes 9.748493194580078 seconds
Save model to ckpts/TransR_DRKG_3
-------------- Test result --------------
Test average MRR : 0.0040337009776222335
Test average MR : 7108.800755156377
Test average HITS@1 : 0.002257297915659451
Test average HITS@3 : 0.0037706748741973485
Test average HITS@10 : 0.0062458718345056756
--------------